# Generate demo content
If not already done:\
Generate demo content (models, use cases, etc.) using the Python script **gen_objects.py** that is included in the Git repository.\
This demo content is required for the code examples below.\
See details in the lab guide **Watsonx.gov-API-Lab-Description.pdf** which is also included in the Git repository.\
(see section "Preparational steps").\
Here is the link to the Git repository:\
https://github.com/andreas-christian/WxGov-API-Lab

# Define your environment settings

In [1]:
# Update the following variable values according to your lab environment!

# Host name of your IBM Cloud Pak for Data cluster
server = "cpd-cpd.apps.6777ccb323fb424f7dff4bfa.ocp.techzone.ibm.com"

# Generate an API authorization token. Link to documentation:
# https://www.ibm.com/docs/en/cloud-paks/cp-data/5.0.x?topic=apis-generating-api-auth-token
token = "YWRtaW46NGlUOEtCam5qODg5Q1QweFBza2pKMWN3TWZYTGpNTHNvbENJOWJwbgo="

# Introduction to REST calls
## Define a request header

In [2]:
import http.client
import json

# First we define a header for the REST calls in this lab
# We use the same header for all REST calls in this notebook
auth_token = "ZenApiKey" + " " + token
header = {
    'Authorization': auth_token,
    'content-type': "application/json",
    'accept': "application/json"
    }
print( "Request Header:")
print( json.dumps(header, indent=4))

Request Header:
{
    "Authorization": "ZenApiKey YWRtaW46NGlUOEtCam5qODg5Q1QweFBza2pKMWN3TWZYTGpNTHNvbENJOWJwbgo=",
    "content-type": "application/json",
    "accept": "application/json"
}


## Execute a simple REST call of type "GET"

The first REST call that we will execute retrieves some configuration infos from the "Governance Console".\
In this example, we retrieve all tags defined in the "Governance Console".

You can also access these tags from the User Interface of the "Governance Console": 
* In the UI open the "Administration menu" (the "gear" icon in the upper right) and then select "Solution Configuration -> Tags"

In [3]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/configuration/tags"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "tags": [
        {
            "tag_id": "1",
            "name": "Input",
            "enabled": "true",
            "created_on": "2025-01-03T19:52:39.170Z",
            "created_by": "OpenPagesAdministrator",
            "modified_date": "2025-01-03T19:52:39.170Z",
            "last_modified_by": "OpenPagesAdministrator"
        },
        {
            "tag_id": "2",
            "name": "Output",
            "enabled": "true",
            "created_on": "2025-01-03T19:52:39.471Z",
            "created_by": "OpenPagesAdministrator",
            "modified_date": "2025-01-03T19:52:39.471Z",
            "last_modified_by": "OpenPagesAdministrator"
        },
        {
            "tag_id": "3",
            "name": "Fairness",
            "enabled": "true",
            "created_on": "2025-01-03T19:52:39.500Z",
            "created_by": "OpenPagesAdministrator",
            "modified_date": "2025-01-03T19:52:39.500Z",
            "last_modified_by": "OpenPag

## Execute a REST call with parameters

Now we execute the same REST call as in the previous example. But we add a parameter "only_enabled"
This will retrieve only those tags that are currently enabled in the system.
A question mark indicates the beginning of the parameters list.
Multiple parameters are separated by ampersand characters (see examples further down)

In [4]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/configuration/tags?only_enabled=true"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "tags": [
        {
            "tag_id": "1",
            "name": "Input",
            "enabled": "true",
            "created_on": "2025-01-03T19:52:39.170Z",
            "created_by": "OpenPagesAdministrator",
            "modified_date": "2025-01-03T19:52:39.170Z",
            "last_modified_by": "OpenPagesAdministrator"
        },
        {
            "tag_id": "2",
            "name": "Output",
            "enabled": "true",
            "created_on": "2025-01-03T19:52:39.471Z",
            "created_by": "OpenPagesAdministrator",
            "modified_date": "2025-01-03T19:52:39.471Z",
            "last_modified_by": "OpenPagesAdministrator"
        },
        {
            "tag_id": "3",
            "name": "Fairness",
            "enabled": "true",
            "created_on": "2025-01-03T19:52:39.500Z",
            "created_by": "OpenPagesAdministrator",
            "modified_date": "2025-01-03T19:52:39.500Z",
            "last_modified_by": "OpenPag

# Useful REST calls to interact with the Governance Console

In the next section, we introduce REST calls that are especially useful for **external systems**
to interact with the *Governance Console*.

Some examples:
* How to retrieve information about "Models" and "Use Cases" from the inventory
* How to retrieve the current status of a model
* How to create new "Models" and "Use Cases" in the inventory
* How to create new "Models" and "Use Cases" and pre-populate some of their fields with data
* How to link existing objects like a "Business Entity" to a model or use case
* How to query information about all currently running workflow instances
* How to query information about specific workflow instances, to check e.g. the current workflow stage

We use two useful methods to process the result set of a REST call:
* json.loads
* json.dumps

The json.loads() method is used to parse a valid JSON string and converts it into a Python Dictionary.
You can then easily access each data element in this dictionary.\
The json.dumps() method converts a Python object into a JSON formatted string.
We use a combination of both methods to pretty print the results of our REST calls.

## Query objects using SQL-like syntax

### Query use cases

Here is an example of a SQL-like query that retrieves the names of all "use cases" stored in the repository: \
SELECT+[Name]+FROM+[Register]

Field names must be embraced with square brackets.\
Special characters like space or percent signs are not allowed in URLs.\
For example, you have to use plus signs instead of spaces.

In [5]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q=SELECT+[Name]+FROM+[Register]"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "definitions": [
        {
            "id": "57",
            "name": "Name",
            "data_type": "STRING_TYPE"
        }
    ],
    "rows": [
        {
            "fields": [
                {
                    "name": "Name",
                    "value": "Applicant_Screening"
                }
            ]
        },
        {
            "fields": [
                {
                    "name": "Name",
                    "value": "Claims_Processing_Automation"
                }
            ]
        },
        {
            "fields": [
                {
                    "name": "Name",
                    "value": "Customer_Support_Chatbots"
                }
            ]
        },
        {
            "fields": [
                {
                    "name": "Name",
                    "value": "Fraud_Detection"
                }
            ]
        },
        {
            "fields": [
                {
                    "name": "N

**Note:** The following query examples contain special characters like spaces,\
single quote and percent sign. These characters are not allowed in URLs.\
Therefore, the replace function is used to replace these characters with characters\
that are allowed in URLs before the queries are executed.

The following query includes a WHERE clause to retrieve use case *Applicant_Screening*.\
It also retrieves additional fields (Resource ID, Description).

In [6]:
query = "SELECT [Resource ID], [Name], [Description] FROM [Register] WHERE [Name] = 'Applicant_Screening'" 
query = query.replace(" ","+").replace("%","%25").replace("'","%27")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "definitions": [
        {
            "id": "28",
            "name": "Resource ID",
            "data_type": "ID_TYPE"
        },
        {
            "id": "57",
            "name": "Name",
            "data_type": "STRING_TYPE"
        },
        {
            "id": "59",
            "name": "Description",
            "data_type": "STRING_TYPE"
        }
    ],
    "rows": [
        {
            "fields": [
                {
                    "name": "Resource ID",
                    "value": "4618"
                },
                {
                    "name": "Name",
                    "value": "Applicant_Screening"
                },
                {
                    "name": "Description"
                }
            ]
        }
    ],
    "offset": 0,
    "limit": 50
}


### Query models
The next example shows a query that contains a LIKE predicate in the search condition and also an ORDER BY clause.\
It retrieves all use cases that contain substring "Classification" in their name.\
The result set is sorted by the use case name.

In [7]:
query = "SELECT [Resource ID], [Name], [Description] FROM [Model] WHERE [Name] LIKE '%Classification%' ORDER BY [Name]" 
# replace special characters in query and convert spaces to plus signs
query = query.replace(" ","+").replace("%","%25").replace("'","%27")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query
conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "definitions": [
        {
            "id": "28",
            "name": "Resource ID",
            "data_type": "ID_TYPE"
        },
        {
            "id": "57",
            "name": "Name",
            "data_type": "STRING_TYPE"
        },
        {
            "id": "59",
            "name": "Description",
            "data_type": "STRING_TYPE"
        }
    ],
    "rows": [
        {
            "fields": [
                {
                    "name": "Resource ID",
                    "value": "4624"
                },
                {
                    "name": "Name",
                    "value": "Applicant_Classification"
                },
                {
                    "name": "Description",
                    "value": "Cluster candidates in high, medium and low profile candidates."
                }
            ]
        },
        {
            "fields": [
                {
                    "name": "Resource ID",
               

### Query models and pretty print the result

The next example shows in more detail how you can access the data returned by a query.\
The query in this example retrieves all *models* from the repository.

The returned data is a string in JSON format.\
We use method **json.loads()** to convert the string into a compound Python data structure.\
This allows us to easily access all elements contained in the data strucure.

This data structure consists of a Python dictionary. Each element of the dictionary\
can either include basic data types like for example strings,\
but also compound data types like dictionaries or arrays.

A good way to understand the structure of a query result set is to first print it out like this:\
**print( json.dumps(json.loads(data), indent=4))**\
This will give you an idea about the general structure of the data. 

In [9]:
# retrieve all "models" from the inventory

query = "SELECT [Resource ID], [Name], [Description] FROM [Model]" 
# replace special characters in query and convert spaces to plus signs
query = query.replace(" ","+").replace("%","%25").replace("'","%27")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    #--------- Access returned data via a Python dictionary
    # Convert JSON string to Python dictionary
    my_dict = json.loads(data)
    # Iterate over rows of result set and print the model names
    print( "ID".ljust(6, ' '), "Name".ljust(40, ' '), "Description")
    print( "".ljust(120, '-'))
    my_array = my_dict["rows"]
    for x in my_array:
        model_id = x["fields"][0]["value"]
        model_name = x["fields"][1]["value"]
        model_description = x["fields"][2]["value"]
        print( model_id.ljust(6, ' '), model_name.ljust(40, ' '), model_description)
        
    print( "\n\nQuery result in JSON format:\n")
    print( json.dumps(json.loads(data), indent=4))
    

Status: 200
Reason: OK

ID     Name                                     Description
------------------------------------------------------------------------------------------------------------------------
2621   MOD_0000000                              Placeholder model used to link to Risk Atlas risk objects
4624   Applicant_Classification                 Cluster candidates in high, medium and low profile candidates.
4625   Credit_Risk_Classification               Cluster customer credits into risk categories.
4626   Customer_Churn_Prediction                Customer churn prediction.
4628   Customer_Segmentation                    Customer_Segmentation


Query result in JSON format:

{
    "definitions": [
        {
            "id": "28",
            "name": "Resource ID",
            "data_type": "ID_TYPE"
        },
        {
            "id": "57",
            "name": "Name",
            "data_type": "STRING_TYPE"
        },
        {
            "id": "59",
            "name": "D

## Retrieve the entire contents of an object

Next we introduce another REST method that retrieves the entire contents of an object (all object fields). As input it requires an object ID.\
In our next example, we first execute a query to retrieve the object ID of use case *Applicant_Screening*:\
SELECT [Resource ID], [Name], [Description] FROM [Register] WHERE [Name] = 'Applicant_Screening'

To find the ID of an object (e.g. use case, model, etc.) you could also access this object in the "Governance Console".\
The object ID can be found at the end of the URL.\
Here is a sample URL (the object ID in this example is 4033):\
https://cpd-cpd.apps.6777ccb323fb424f7dff4bfa.ocp.techzone.ibm.com/openpages-openpagesinstance-cr/app/jspview/react/grc/task-view/4033

In [10]:
query = "SELECT [Resource ID], [Name], [Description] FROM [Register] WHERE [Name] = 'Applicant_Screening'" 
query = query.replace(" ","+").replace("%","%25").replace("'","%27")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/query?q="+query
conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    # Retrieve object ID from query result
    data = response.read().decode("utf-8")
    #print( json.dumps(json.loads(data), indent=4))
    my_dict = json.loads(data)
    use_case_id = my_dict["rows"][0]["fields"][0]["value"]
    use_case_name = my_dict["rows"][0]["fields"][1]["value"]
    print("use_case_id=",use_case_id)
    print("use_case_name=",use_case_name)

Status: 200
Reason: OK

use_case_id= 4618
use_case_name= Applicant_Screening


Now we retrieve the entire object with all its fields using the following endpoint:\
**https://{server_name}/{basePath}/v2/contents/{id}**

In [11]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/contents/"+use_case_id

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "name": "Applicant_Screening",
    "id": "4618",
    "path": "/_op_sox/Project/Default/ICDocumentation/Registers/My_Enterprise/HR_Department/Applicant_Screening.txt",
    "creation_date": "2025-02-19T10:26:35.000Z",
    "modified_date": "2025-02-23T22:45:26.000Z",
    "parent_folder_id": "4563",
    "creator": "admin",
    "last_modified_by": "admin",
    "fields": [
        {
            "id": "28",
            "data_type": "ID_TYPE",
            "name": "Resource ID",
            "has_changed": false,
            "value": "4618"
        },
        {
            "id": "65",
            "data_type": "STRING_TYPE",
            "name": "Comment",
            "has_changed": false
        },
        {
            "id": "62",
            "data_type": "INTEGER_TYPE",
            "name": "Created By",
            "has_changed": false,
            "value": 1002
        },
        {
            "id": "61",
            "data_type": "DATE_TYPE",
            "name": "

## Update the fields of an object

Next, we update some fields of use case *Applicant_Screening*.\
The corresponding *use_case_id* and *use_case_name* have already been retrieved in the previous steps (see above).

In [13]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/contents/"+use_case_id
request_type = "PUT"

payload = {
    "type_definition_id": "74",
    "name": use_case_name,
    "description": "New Description GRC Object via Update V2",
    "fields":[
               {
            "name": "MRG-AIFacts-ModelUseCase:Purpose",
            "value": "My new purpose description"
        }
    ]
}
payload = {
    "type_definition_id": "74",
    "name": "Applicant_Screening",
}
# Convert dictionary to JSON string
print(endpoint)
print(payload)
payload = json.dumps(payload)
print(use_case_id)

# Execute REST call
conn = http.client.HTTPSConnection(server)
conn.request(request_type, endpoint, payload, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))

if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

/openpages-openpagesinstance-cr-opgrc/api/v2/contents/4618
{'type_definition_id': '74', 'name': 'Applicant_Screening'}
4618


Status: 200
Reason: OK

{
    "name": "Applicant_Screening",
    "id": "4618",
    "path": "/_op_sox/Project/Default/ICDocumentation/Registers/My_Enterprise/HR_Department/Applicant_Screening.txt",
    "creation_date": "2025-02-19T10:26:35.000Z",
    "modified_date": "2025-02-24T09:39:07.000Z",
    "parent_folder_id": "4563",
    "creator": "admin",
    "last_modified_by": "admin",
    "fields": [
        {
            "id": "28",
            "data_type": "ID_TYPE",
            "name": "Resource ID",
            "has_changed": false,
            "value": "4618"
        },
        {
            "id": "65",
            "data_type": "STRING_TYPE",
            "name": "Comment",
            "has_changed": false
        },
        {
            "id": "62",
            "data_type": "INTEGER_TYPE",
            "name": "Created By",
            "has_changed": false,
            "value": 1002
        },
        {
            "id": "61",
            "data_type": "DATE_TYPE",
            "name": "

## Create new objects

The next REST call creates an object of type *Model* in the inventory of the *Governance console*.
It also links this object to a parent object of type *Business Entity*. The ID of the parent object in this example is 4035. 
We also show below how to retrieve the ID of the new model from the REST call response.

In addition to some basic fields like "name" and "description", we also define the following additional fields of the model:
* Field "Model Owner" = "admin"
* Field "Model or Non-Model" = "Model"
* Field "Machine Learning Model" = "Yes"

After you have executed this REST call, you can go to the *Governance Console*, 
open the main menu and select "Inventory -> Models" and search for "Customer_Segmentation".

**Note:**\
When you create an object (for example a *model* or a *use case*) via the REST API, the corresponding workflow is automatically started, if workflow "Type" is set to "Auto start".
After you have executed the below REST call, workflow "Model Candidate" will automatically start. Check that all mandatory fields are defined in the object, otherwise the corresponding workflow may not start. Here are the steps to find the workflow instance related to the new model in the Governance Console:
* Click "Open Administration menu" (the "gear" icon located in the upper right of the Governance Console)
* Click "Solution Configuration -> Workflows"
* Click "Workflow Instances"
* In column "Name" you will find the name of your new model (in the below example, we create a model with name 'Customer_Segmentation'.

**Note:**\
Before you include references to other objects in a REST call, for example,
a reference to a parent object, you should first verify that those objects really exist.
This can be done via the *Governance console*. 
* For example, to verify that a specific business entity exists
you can open the "Governance console", then open the main menu and
select "Organisation -> Business Entities".
* To check if a certain user exists and to see the ID of the user, in the "Governance console"
you open the administration menu and select "Users and Security -> Users" and then click
on a user name. You will see the detailed user profile. The ID of the user can be found in the URL
of your web browser. For example, in the following URL the user's ID is "1002":\
https://cpd-cpd.apps.6777ccb323fb424f7dff4bfa.ocp.techzone.ibm.com/openpages-openpagesinstance-cr/app/jspview/react/grc/admin/users-edit/1002

### Create a new "business entity"

In [15]:
endpoint= "/openpages-openpagesinstance-cr-opgrc/api/v2/contents"
request_type = "POST"
name = 'Your_Enterprise'

payload = {
        'name': name,
        'description': name,
        "creator": 'admin',
        'type_definition_id': '5',
}
# Convert dictionary to JSON string
payload = json.dumps(payload)
# Execute REST call
conn = http.client.HTTPSConnection(server)
conn.request(request_type, endpoint, payload, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))

if response.status == 201:
    data = response.read().decode("utf-8")
    # Now we retrieve the ID of the new object from the REST call result
    my_dict = json.loads(data)
    object_id = my_dict["id"]
    print( "ID of the new object: {}\n\n".format( object_id))  
business_entity = object_id

Status: 201
Reason: Created

ID of the new object: 4628


{
    "name": "Customer_Segmentation",
    "id": "4628",
    "path": "/_op_sox/Project/Default/ICDocumentation/Models/Your_Enterprise/Customer_Segmentation.txt",
    "creation_date": "2025-02-20T15:12:51.000Z",
    "modified_date": "2025-02-24T09:40:04.000Z",
    "description": "Customer_Segmentation",
    "parent_folder_id": "4578",
    "creator": "admin",
    "last_modified_by": "admin",
    "fields": [
        {
            "id": "28",
            "data_type": "ID_TYPE",
            "name": "Resource ID",
            "has_changed": false,
            "value": "4628"
        },
        {
            "id": "65",
            "data_type": "STRING_TYPE",
            "name": "Comment",
            "has_changed": false
        },
        {
            "id": "62",
            "data_type": "INTEGER_TYPE",
            "name": "Created By",
            "has_changed": false,
            "value": 1002
        },
        {
            "id"

### Create a new model

In [15]:
# now create a new model and assign the business entity that was created before as parent object
endpoint= "/openpages-openpagesinstance-cr-opgrc/api/v2/contents"
request_type = "POST"
name = 'Customer_Segmentation'

payload = {
        'name': name,
        'description': name,
        'type_definition_id': '58',
        'primary_parent_id': business_entity,
        'title': name,
 'fields': [
        {
            "id": "1275",
            "data_type": "STRING_TYPE",
            "name": "MRG-Model:Model Owner",
            "value": "admin"
        },
        {
            'id': '1286',
            'data_type': 'ENUM_TYPE',
            'name': 'MRG-Model:Model Non Model',
             'value': {
                'id': '3696',
                'name': 'Model',
                'localized_label': 'Model',
                'index': 1,
             }
        },
	    {
            'id': '1279',
            'data_type': 'ENUM_TYPE',
            'name': 'MRG-Model:Machine Learning Model',
             'value': {
                'id': '3694',
                'name': 'Yes',
                'localized_label': 'Yes',
                'index': 1,
              }
        }
]
}
# Convert dictionary to JSON string
payload = json.dumps(payload)
# Execute REST call
conn = http.client.HTTPSConnection(server)
conn.request(request_type, endpoint, payload, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))

if response.status == 201:
    data = response.read().decode("utf-8")
    # Now we retrieve the ID of the new object from the REST call result
    my_dict = json.loads(data)
    object_id = my_dict["id"]
    print( "ID of the new object: {}\n\n".format( object_id))
    # Now we print the entire result in JSON format
    print( json.dumps(json.loads(data), indent=4))

Status: 201
Reason: Created

ID of the new object: 4628


{
    "name": "Customer_Segmentation",
    "id": "4628",
    "path": "/_op_sox/Project/Default/ICDocumentation/Models/Your_Enterprise/Customer_Segmentation.txt",
    "creation_date": "2025-02-20T15:12:51.000Z",
    "modified_date": "2025-02-24T09:40:04.000Z",
    "description": "Customer_Segmentation",
    "parent_folder_id": "4578",
    "creator": "admin",
    "last_modified_by": "admin",
    "fields": [
        {
            "id": "28",
            "data_type": "ID_TYPE",
            "name": "Resource ID",
            "has_changed": false,
            "value": "4628"
        },
        {
            "id": "65",
            "data_type": "STRING_TYPE",
            "name": "Comment",
            "has_changed": false
        },
        {
            "id": "62",
            "data_type": "INTEGER_TYPE",
            "name": "Created By",
            "has_changed": false,
            "value": 1002
        },
        {
            "id"

### Create other types of objects
To create other types of objects, you need to specify their corresponding **type_definition_id** in the REST call.\
For example, a "use case" has type_definition_id 74. Hence you would use the following payload:
<pre>
payload = {
        'name': 'Some text',
        'description': 'Some text',
        'type_definition_id': '74',
        'primary_parent_id': 'Some ID',
        'title': 'Some text'
}
</pre>

Here are the type definition IDs of frequently used object types:
* Use Case: 74
* Model: 58
* Business Entity: 5
* File (e.g. document): 4
* ChangeRequest: 33
* Attestation: 24
* Issue: 6
* Action Item: 7

The following REST call retrieves all **type names** and the corresponding **type IDs** from the *Governance Console*.

In [16]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/types"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    #print( json.dumps(json.loads(data), indent=4))
    my_dict = json.loads(data)
    my_array = my_dict["types"]
    print( "Type ID".ljust(10, ' '), "Type Name".ljust(30, ' '), "Localized Label")
    print( "---------------------------------------------------------------------")
    for x in my_array:
        type_id = x["id"]
        type_name = x["name"]
        localized_label = x["localized_label"]
        print( type_id.ljust(10, ' '),type_name.ljust(30, ' '),localized_label)
        #print( type_id.ljust(10, ' '),';',type_name.ljust(30, ' '),';',localized_label)


Status: 200
Reason: OK

Type ID    Type Name                      Localized Label
---------------------------------------------------------------------
23         Assertion                      Assertion
24         Attestation                    Attestation
27         AuditPhase                     Audit Section
25         AuditProgram                   Audit
28         AuditableEntity                Auditable Entity
29         Auditor                        Auditor
108        BCBusinessImpactAnalysis       Business Impact Analysis
109        BCEvent                        Business Continuity Event
110        BCPlan                         Business Continuity Plan
111        BCTest                         Business Continuity Test Plan
112        BCTestResult                   Business Continuity Test Result
115        BusService                     Business Service
116        BusServiceEval                 Business Service Eval
31         Campaign                       Campaign
32     

The next REST call retrieves all supported **field names** of objects of type **model** (type id = 58).

In [17]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/types/58"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    # print( json.dumps(json.loads(data), indent=4))
    my_dict = json.loads(data)
    my_array = my_dict["field_definitions"]
    print( "Field Name".ljust(50, ' '), "Data Type")
    print( "---------------------------------------------------------------------")
    for x in my_array:
        type_name = x["name"]
        data_type = x["data_type"]
        print( type_name.ljust(50, ' '),data_type)


Status: 200
Reason: OK

Field Name                                         Data Type
---------------------------------------------------------------------
Resource ID                                        ID_TYPE
Comment                                            STRING_TYPE
Created By                                         INTEGER_TYPE
Creation Date                                      DATE_TYPE
Description                                        STRING_TYPE
Last Modification Date                             DATE_TYPE
Last Modified By                                   INTEGER_TYPE
Location                                           STRING_TYPE
Name                                               STRING_TYPE
Orphan                                             STRING_TYPE
Title                                              STRING_TYPE
MRG-Model:Additional Description                   LARGE_STRING_TYPE
MRG-Model:Approval Closure Date                    DATE_TYPE
MRG-Model:Approval Expected 

## Retrieve information about workflows

### Get currently running workflows related to models

In [27]:
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?type_definition=Model"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
print('Model'.ljust(8, ' '),'Model Name'.ljust(30, ' '),'Workflow instance ID and name'.ljust(38, ' '),'Workflow stage')
print("--------------------------------------------------------------------------------------------------------------------")
if response.status == 200:
    data = response.read().decode("utf-8")
    # Extract/print most important infos from REST call response
    my_dict = json.loads(data)
    my_array = my_dict["activity_instances"]
    for x in my_array:
        workflow_instance_id = x["id"]
        workflow_name = x["workflow"]["name"]
        workflow_stage = x["status"]
        model_id = x["grc_object"]["id"]
        model_name = x["grc_object"]["name"]
        print(model_id.ljust(8, ' '),model_name.ljust(30, ' '),workflow_instance_id.ljust(8, ' '),workflow_name.ljust(30, ' '),workflow_stage)
    # Print result in JSON format
    print( "\n\n", json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

Model    Model Name                     Workflow instance ID and name          Workflow stage
--------------------------------------------------------------------------------------------------------------------
4624     Applicant_Classification       142      Model Candidate                Propose Candidate
4625     Credit_Risk_Classification     143      Model Candidate                Propose Candidate
4626     Customer_Churn_Prediction      144      Model Candidate                Propose Candidate
4628     Customer_Segmentation          145      Model Candidate                Propose Candidate


 {
    "activity_instances": [
        {
            "id": "142",
            "status": "Propose Candidate",
            "criticality": "medium",
            "due_date": "",
            "assignees": [
                "admin"
            ],
            "process_start_date": "2/19/2025",
            "process_due_date": "2/26/2025",
            "grc_object": {
           

### Different search criteria to retrieve workflow instances

In [28]:
# Retrieve workflow instances related to "Use Cases" (type "Register" is related to "Use Cases")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?type_definition=Register"
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?type_definition=74"

# Retrieve workflow instances related to "Models"
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?type_definition=Model"

# Retrieve all instances related to a specific workflow , e.g. workflow ID = 161
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?workflow=161"

# Retrieve currently running workflows which are assigned to a specific user
#endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/instances/search?user_role=assignee&user=admin"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "activity_instances": [
        {
            "id": "137",
            "status": "Stage 2",
            "criticality": "medium",
            "due_date": "",
            "process_start_date": "2/19/2025",
            "process_due_date": "",
            "grc_object": {
                "id": "4619",
                "name": "Claims_Processing_Automation",
                "path": "/_op_sox/Project/Default/ICDocumentation/Registers/My_Enterprise",
                "type_definition": {
                    "id": "74",
                    "name": "Register",
                    "label": "Use Case"
                }
            },
            "workflow": {
                "id": "161",
                "name": "Sample workflow",
                "label": "Sample workflow",
                "version": "5",
                "activity": {
                    "id": "846",
                    "name": "Stage 2",
                    "label": "Stage 2"
                }
         

### Retrieve information about workflow definitions

The next example shows how to retrieve information about the components of a workflow:
* Workflow stages
* Workflow actions

In this example, we retrieve the definitions of all workflows of type "Use Case".

In [29]:
# Retrieve workflow definitions related to "Use Cases" (type "Register" is related to "Use Cases")
endpoint = "/openpages-openpagesinstance-cr-opgrc/api/v2/workflows/definitions?object_type=Register"

conn = http.client.HTTPSConnection(server)
conn.request("GET", endpoint, headers=header)
response = conn.getresponse()
print("Status: {}\nReason: {}\n".format(response.status, response.reason))
if response.status == 200:
    data = response.read().decode("utf-8")
    print( json.dumps(json.loads(data), indent=4))

Status: 200
Reason: OK

{
    "definitions": [
        {
            "id": "59",
            "name": "Model Use Case Request",
            "label": "Model Use Case Request",
            "version_id": "110",
            "type_definition_name": "Register",
            "type": "create",
            "version_number": "3",
            "latest_version_number": "4",
            "state": "published",
            "last_version_state": "draft",
            "is_automated": false,
            "is_enabled": false,
            "last_modified_by": "admin",
            "modified_on": "2025-01-09T15:25:36.177Z",
            "created_by": "OpenPagesAdministrator",
            "created_on": "2024-09-06T15:41:27.823Z",
            "activities": [
                {
                    "name": "Mandate Selection",
                    "label": "Mandate Selection",
                    "transitions": [
                        {
                            "name": "Submit for Approval",
                        